In [1]:
import parmed
import numpy as np
import itertools

from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

# Load solvated protein

In [69]:
#Load up CHARMM-GUI pdb, and create a PARMED structure from it.  
prot_pdb = PDBFile('./1xep-processed.pdb')

# Delete waters to save as receptor alone prmtop:

In [70]:
mod = Modeller(prot_pdb.topology, prot_pdb.positions)

In [71]:
chains = [i for i in mod.topology.chains()]
for count, ch in enumerate(chains):
    print(f'\nChain {count}:')
    atoms = [i for i in ch.atoms()]
    print('\t', atoms[:5])
    print('\t', atoms[-5:])


Chain 0:
	 [<Atom 0 (N) of chain 0 residue 0 (MET)>, <Atom 1 (H) of chain 0 residue 0 (MET)>, <Atom 2 (H2) of chain 0 residue 0 (MET)>, <Atom 3 (H3) of chain 0 residue 0 (MET)>, <Atom 4 (CA) of chain 0 residue 0 (MET)>]
	 [<Atom 2596 (NZ) of chain 0 residue 161 (LYS)>, <Atom 2597 (HZ1) of chain 0 residue 161 (LYS)>, <Atom 2598 (HZ2) of chain 0 residue 161 (LYS)>, <Atom 2599 (HZ3) of chain 0 residue 161 (LYS)>, <Atom 2600 (OXT) of chain 0 residue 161 (LYS)>]

Chain 1:
	 [<Atom 2601 (O) of chain 1 residue 162 (HOH)>, <Atom 2602 (H1) of chain 1 residue 162 (HOH)>, <Atom 2603 (H2) of chain 1 residue 162 (HOH)>, <Atom 2604 (O) of chain 1 residue 163 (HOH)>, <Atom 2605 (H1) of chain 1 residue 163 (HOH)>]
	 [<Atom 18406 (H1) of chain 1 residue 5430 (HOH)>, <Atom 18407 (H2) of chain 1 residue 5430 (HOH)>, <Atom 18408 (O) of chain 1 residue 5431 (HOH)>, <Atom 18409 (H1) of chain 1 residue 5431 (HOH)>, <Atom 18410 (H2) of chain 1 residue 5431 (HOH)>]

Chain 2:
	 [<Atom 18411 (Na) of chain 2 res

In [72]:
#looks like chain 1 and 2 contain water and ions. So let's delete those:
mod.delete([chains[1]])
chains = [i for i in mod.topology.chains()] #dunno why but you have to rewrite the chains
                                            #ref here in order to delete the right one.
mod.delete([chains[1]])

In [73]:
omm_forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')
prot_system = omm_forcefield.createSystem(mod.topology, rigidWater=False)
prot_structure = parmed.openmm.load_topology(mod.topology,
                                           prot_system,
                                           xyz=mod.positions)

In [74]:
prot_structure.save('receptor.prmtop')

# Same again with the complex:

In [77]:
#Load up the parameterized drug system, and again make it into a parmed structure:
drug_system = XmlSerializer.deserialize(open('./drug_system.xml').read())
drug_pdbfile = PDBFile('./catechol_aligned.pdb')

drug_structure = parmed.openmm.load_topology(drug_pdbfile.topology,
                                                drug_system,
                                                xyz=drug_pdbfile.positions)

In [78]:
complex_no_water = prot_structure + drug_structure

In [79]:
complex_no_water.save('complex.prmtop')